# ISE529 Predictive Analytics
**Homework #4**

**Student Name:** `André Ramolivaz` <br>
**NetID:** `3933665317` <br>
**Due Date:** `2025-06-17`

### Setup
Import all required packages below.

In [31]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

---
## Question 1
*25 points*
![Logo Python](16.png)


**(a)**

In [12]:
df = pd.read_csv('Default.csv')

df['default_binary'] = (df['default'].str.lower() == 'yes').astype(int)

X = sm.add_constant(df[['income', 'balance']])
y = df['default_binary']

model = sm.GLM(y, X, family=sm.families.Binomial()).fit()

results_df = pd.DataFrame({
    'Coefficient': model.params,
    'Std. Error': model.bse
})

print(model.summary())
results_df

                 Generalized Linear Model Regression Results                  
Dep. Variable:         default_binary   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Sun, 15 Jun 2025   Deviance:                       1579.0
Time:                        09:33:30   Pearson chi2:                 6.95e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1256
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.0

,Coefficient,Std. Error
const,-11.540468,0.434772
income,0.000021,0.000005
balance,0.005647,0.000227


| Term        | Estimate      | Std. Error | z-value | p-value |
|-------------|---------------|------------|---------|---------|
| Intercept   | −11.5405      | **0.4348** | −26.54  | < 0.001 |
| income      | 2.08 × 10⁻⁵   | **4.99 × 10⁻⁶** |  4.17   | < 0.001 |
| balance     | 0.00565       | **2.27 × 10⁻⁴** | 24.84   | < 0.001 |


* **income** – A one-dollar increase in annual income (holding balance constant) raises the log-odds of default by roughly 2.08 × 10⁻⁵: statistically significant but economically small.
* **balance** – Each additional dollar of credit-card balance increases the log-odds of default by about 0.00565, a much stronger effect.
* The tiny standard errors reflect the precision afforded by the large sample (n = 10 000).


**(b)**

In [6]:
def boot_fn(data: pd.DataFrame, idx: np.ndarray) -> np.ndarray:
    sample = data.iloc[idx].copy()

    sample["default_bin"] = (sample["default"].str.lower() == "yes").astype(int)

    X = sm.add_constant(sample[["income", "balance"]])
    y = sample["default_bin"]

    model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
    return model.params[["income", "balance"]].values

**(c)**

In [8]:
B = 1000
n = len(df)
np.random.seed(0)

boot_coefs = np.empty((B, 2))
for b in range(B):
    indices = np.random.choice(n, n, replace=True)
    boot_coefs[b] = boot_fn(df, indices)

# Standard errors from bootstrap
se_income, se_balance = boot_coefs.std(axis=0, ddof=1)
print(f"SE income   : {se_income:.10e}")
print(f"SE balance  : {se_balance:.10e}")


SE income   : 4.7155561618e-06
SE balance  : 2.3628201083e-04


**(d)**

Both approaches give almost identical standard errors: the differences are ≲ 5 %, well within the Monte-Carlo noise you expect when bootstrapping only 1 000 resamples.
The analytical SEs from sm.GLM() come from the large-sample (Fisher-information) theory of maximum-likelihood estimators. With n = 10 000 and a correctly specified model (logit link, independent observations), those asymptotic formulas are already very accurate, so resampling adds little new information.

For income the bootstrap SE is slightly smaller; for balance slightly larger. Such tiny shifts are typical random variation—if increase B or change the seed, the gap will fluctuate around zero.

Bootstrap is most useful when the sample is small, the likelihood assumptions are doubtful (heavy tails, dependence, heteroskedasticity), or when is needed SEs for statistics that lack simple formulas.

---
## Question 2
*25 points*
![Logo Python](17.png)

**(i)**

In [18]:
weekly = pd.read_csv('Weekly.csv')

weekly["Dir_bin"] = (weekly["Direction"].str.lower() == "up").astype(int)
def loocv_fit_without_i(data: pd.DataFrame, i: int) -> sm.GLM:

    train_mask = data.index != i

    X_train = sm.add_constant(
        data.loc[train_mask, ["Lag1", "Lag2"]],
        has_constant="add"
    )
    y_train = data.loc[train_mask, "Dir_bin"]

    model_i = sm.GLM(y_train, X_train, family=sm.families.Binomial()).fit()

    return model_i

# Example: fit the model leaving out the 42-nd observation
model_42 = loocv_fit_without_i(weekly, i=42)
model_42.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Dir_bin   No. Observations:                 1088
Model:                            GLM   Df Residuals:                     1085
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -743.46
Date:                Sun, 15 Jun 2025   Deviance:                       1486.9
Time:                        09:58:05   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007418
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2195      0.061      3.570      0.000       0.099       0.340
Lag1          -0.0394      0.026     -1.502      0.133      -0.091       0.012
Lag2           0.0605      0.027      2.277      0.023       0.008       0.113
==============================================================================
"""

**(ii)**

In [19]:
def loocv_prob_up_i(model_i: sm.GLM, data: pd.DataFrame, i: int) -> float:

    X_test = sm.add_constant(
        data.loc[[i], ["Lag1", "Lag2"]],
        has_constant="add"
    )

    prob_up = model_i.predict(X_test).iloc[0]
    return float(prob_up)

# Example usage for observation 42
model_42 = loocv_fit_without_i(weekly, i=42)

# 2. Predict P(Up) for row 42 ← step (ii)
p_up_42 = loocv_prob_up_i(model_42, weekly, i=42)
print(f"Predicted P(Up) for observation 42: {p_up_42:.3f}")

Predicted P(Up) for observation 42: 0.523


**(iii)**

In [21]:
def loocv_error_i(i: int, data) -> int:

    model_i = loocv_fit_without_i(data, i)      # ← fixed: data first, i second

    p_up = loocv_prob_up_i(model_i, data, i)

    y_hat = 1 if p_up > 0.5 else 0          # predicted class
    err_i = int(y_hat != data.loc[i, "Dir_bin"])
    return err_i

# Example: compute Err_42  (0 = correct, 1 = misclassified)

err_42 = loocv_error_i(42, weekly)
print(f"Err_42 = {err_42}")

Err_42 = 0


**(iv)**

In [23]:
n = len(weekly)

probs   = np.empty(n)
y_hat   = np.empty(n, int)
y_true  = weekly["Dir_bin"].to_numpy()
errors  = np.empty(n, int)

for i in range(n):
    model_i   = loocv_fit_without_i(weekly, i)     # step (i)
    probs[i]  = loocv_prob_up_i(model_i, weekly, i)# step (ii)
    y_hat[i]  = int(probs[i] > 0.5)                # step (iii)
    errors[i] = int(y_hat[i] != y_true[i])         # step (iii)

CV_n = errors.mean()

loocv_df = (
    pd.DataFrame({
        "Prob_Up": probs,
        "Pred": y_hat,
        "Actual": y_true,
        "Err_i": errors
    })
    .assign(Row=lambda d: d.index)
    .set_index("Row")
)
display(loocv_df.head())

print(f"\nTotale errori   : {errors.sum()} su {n}")
print(f"LOOCV error rate: {CV_n:.3f}")



,Prob_Up,Pred,Actual,Err_i
Row,,,,
0,0.571392,1,0,1
1,0.570339,1,0,1
2,0.574645,1,1,0
3,0.480442,0,1,1
4,0.598801,1,1,0



Totale errori   : 490 su 1089
LOOCV error rate: 0.450


---
## Question 3
*25 points*
![Logo Python](18.png)

In [25]:
df = pd.read_csv('Carseats.csv')

df_enc = pd.get_dummies(
    df,
    columns=['ShelveLoc', 'Urban', 'US'],
    drop_first=True
)

X = df_enc.drop('Sales', axis=1)
y = df_enc['Sales']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=1
)

rf = RandomForestRegressor(
    n_estimators=500,     # 500 bootstrap trees
    random_state=1,       # reproducibility of bootstrap samples & feature splits
    bootstrap=True,       # default; each tree gets a bootstrap sample of rows
    oob_score=False       # OOB error not used here
)
rf.fit(X_train, y_train)


y_train_pred = rf.predict(X_train)
y_test_pred  = rf.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse  = mean_squared_error(y_test,  y_test_pred)

print(f"Training MSE: {train_mse:.3f}")
print(f"Test MSE    : {test_mse:.3f}")


importances = pd.Series(
    rf.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

print("\nTop 10 features by importance:")
print(importances.head(10))

Training MSE: 0.327
Test MSE    : 2.852

Top 10 features by importance:
Price               0.294173
ShelveLoc_Good      0.232180
Age                 0.102342
CompPrice           0.096213
Advertising         0.077246
ShelveLoc_Medium    0.070332
Income              0.047548
Population          0.043278
Education           0.027431
Urban_Yes           0.005562
dtype: float64


---
## Question 4
*25 points*
![Logo Python](19.png)

**(a)**

In [41]:
caravan = pd.read_csv("Caravan.csv")

train_df = caravan.iloc[:1000].copy()
test_df  = caravan.iloc[1000:].copy()

print(f"Training set shape: {train_df.shape}")
print(f"Test set shape    : {test_df.shape}")

Training set shape: (1000, 86)
Test set shape    : (4822, 86)


**(b)**

In [42]:
X_train = train_df.drop(columns=["Purchase"])
X_test  = test_df.drop(columns=["Purchase"])

le = LabelEncoder()
y_train_bin = le.fit_transform(train_df["Purchase"])  # Yes→1, No→0
y_test_bin  = le.transform(test_df["Purchase"])


gb = GradientBoostingClassifier(
        n_estimators=1000,
        learning_rate=0.01,
        max_leaf_nodes=5,        # ≈ 4 split for tree
        random_state=1
)
gb.fit(X_train, y_train)

importances = (
    pd.Series(gb.feature_importances_, index=X_train.columns)
      .sort_values(ascending=False)
)
print("\nTop predictors by importance:")
print(importances.head(15))


Top predictors by importance:
MOSTYPE     0.075484
PPERSAUT    0.074459
MGODGE      0.066166
ABRAND      0.055200
MKOOPKLA    0.045845
MBERMIDD    0.043912
PBRAND      0.040271
AMOTSCO     0.039088
APLEZIER    0.034719
PPLEZIER    0.034538
MFALLEEN    0.031596
MOPLHOOG    0.029480
PLEVEN      0.029348
MBERARBG    0.023757
MSKB1       0.023365
dtype: float64


**(c)**

In [43]:
probs  = gb.predict_proba(X_test)[:, 1]      # P(Purchase = "Yes")
y_pred = (probs > 0.20).astype(int)

cm = confusion_matrix(y_test_bin, y_pred, labels=[1, 0])
cm_df = pd.DataFrame(
    cm,
    index=["Actual: Yes (1)", "Actual: No (0)"],
    columns=["Predicted: Yes (1)", "Predicted: No (0)"]
)

print("\nConfusion matrix (threshold = 20%):")
print(cm_df)


Confusion matrix (threshold = 20%):
                 Predicted: Yes (1)  Predicted: No (0)
Actual: Yes (1)                  39                250
Actual: No (0)                  181               4352
